In [70]:
import boto3
import json
from jinja2 import Environment, FileSystemLoader
import sys
from pathlib import Path
import os

In [71]:
template_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts-templates"))
output_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","model-results"))
ontology_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","ontologies"))
ontology_name = "PropertyAppraisalOntology-v2.ttl"
input_text_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts"))
input_text_name = "Appraisal-Report-address.txt"
template_filename = "prompt-template-ontology-with-example-modified.txt"
output_name = "appraisal-report-addr-updated-example"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

In [72]:
bedrock_runtime= boto3.client("bedrock-runtime",region_name="us-east-1")

In [73]:
environment = Environment(loader=FileSystemLoader(template_dir))
template = environment.get_template(template_filename)

In [74]:
with open(os.path.join(ontology_dir,ontology_name),"r") as f:
    ontology = f.read()

In [75]:
with open(os.path.join(input_text_dir,input_text_name),"r") as f:
    input_text = f.read()

In [76]:
prompt = template.render(
    ontology=ontology,
    input_text=input_text
)

In [77]:
prompt

'You are a faithful and trusted network graph maker tasked with creating a comprehensive and interconnected representation of a given context provided between the <input> and </input> tags below\nensuring all nodes are part of a single cluster using the turtle ontology provided between the <ontology> and </ontology> tags. \nYour job  involves to reason step by step and extract key concepts (nodes) and their relationships (links) \nfrom the provided text and constructing a JSON.\n\nWhile doing so, follow these nine guidelines below faithfully:\n\n1. Nodes Identification: Identify entities in the input text. Align these entities to classes and attributes defined in the ontology\n\n2. Relationship Mapping: Establish clear relationships between all nodes using the definitions provided in the relationships section of the ontology.\n\n3. Relationship Validation: Confirm each relationship\'s validity and relevance based on the text. Avoid introducing external concepts not present in the conte

In [78]:
message={"role": "user","content": prompt}

In [79]:
messages = [message]

In [80]:
body=json.dumps(
{
    'messages':messages,
    'anthropic_version':'bedrock-2023-05-31',
    'top_p':0.5,
    'temperature':0.5,
    'top_k':5,
    'max_tokens':3000
})

In [81]:
response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType='application/json',
    accept='application/json',
    body=body
)

In [82]:
response_body = json.loads(response['body'].read())

In [83]:
response_body

{'id': 'msg_01VnjUWXN1KshW7AQuifsVWx',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': '```json\n{\n    "nodes": [\n        {\n            "id": "Knowledge",\n            "attributes": "",\n            "type": "concept",\n            "group": 1\n        },\n        {\n            "id": "PropertyAppraisal",\n            "attributes": "",\n            "type": "document",\n            "group": 2\n        },\n        {\n            "id": "CollegeCourtyard",\n            "attributes": "PropertyName",\n            "type": "entity",\n            "group": 3\n        },\n        {\n            "id": "RaiderHousing",\n            "attributes": "PropertyName",\n            "type": "entity",\n            "group": 3\n        },\n        {\n            "id": "62Units",\n            "attributes": "PropertyUnits",\n            "type": "entity",\n            "group": 4\n        },\n        {\n            "id": "MultiFamily",\n            "attributes": "PropertyBuildin

In [84]:
full_output_name = f"{model_id}-{template_filename.split('.txt')[0]}-{output_name}"
full_output_name=full_output_name.replace(":","-")
same_name_count = int(len([f for f in os.listdir(output_dir) if full_output_name in f])/2)
file_count = "" if same_name_count ==0 else f'-{same_name_count}'

In [85]:
full_output_name

'anthropic.claude-3-sonnet-20240229-v1-0-prompt-template-ontology-with-example-modified-appraisal-report-addr-updated-example'

In [86]:
with open(os.path.join(output_dir,f"{full_output_name}{file_count}.json"),"w+") as f:
    json.dump(response_body,f)

In [87]:
with open(os.path.join(output_dir,f"{full_output_name}-graph{file_count}.json"),"w+") as f:
    json.dump(json.loads(response_body["content"][0]["text"].split("```")[1].split("json")[1]),f)

JSONDecodeError: Unterminated string starting at: line 340 column 13 (char 10556)